<a href="https://colab.research.google.com/github/takatakamanbou/MVA/blob/2023/ex14notebookC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MVA2023 ex13notebookC

<img width=64 src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/MVA/MVA-logo13.png"> https://www-tlab.math.ryukoku.ac.jp/wiki/?MVA/2023

In [ ]:
# いつものいろいろインポート
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn
seaborn.set()

# SciPy の DCT と FFT の関数
from scipy.fft import dct, fft, ifft, fftfreq

# 科学技術計算のライブラリ SciPy の中の WAVE ファイルを扱うモジュール
from scipy.io.wavfile import read, write

# コンピュータビジョン・画像処理のライブラリ OpenCV のモジュールをインポート
import cv2

# notebook で画像を表示するためのほげ
from IPython.display import display, Image

----
## 【よだんだよん】2次元DCTと画像圧縮
----

この notebook は発展的な話題を扱っています．

notebookA で解説したDCTは，平面上で2方向に広がる波を表すように2次元へと拡張できます．グレイスケール画像（※注）の画素値は縦横に広がった波でできていると考えることができるので，2次元のDCTを適用することができます．

<span style="font-size: 75%">
※注: 色を含まない画像のことです．カラー画像の場合について気になるひとは，takataka に尋ねてね．
</span>


---
### 準備あれこれ

いくつか関数を定義しときます．

In [ ]:
### 画像表示用の関数
#
def displayImage(img):

    if type(img) is np.ndarray:              # 渡されたのが np.ndarray だった場合（OpenCVの画像はこの形式)
        rv, buf = cv2.imencode('.png', img)  # PNG形式のバイト列に変換
        if rv:
            display(Image(data=buf.tobytes()))   # 変換できたらバイト列を渡して表示
            return
    elif type(img) is str:                   # 渡されたのが文字列だった場合
        display(Image(filename=img))       # それがファイル名だと思って読み込む
        return

    print('displayImage: error')

In [ ]:
### 2D-DCTの基底を作る関数
#
def getDCTbasis2D(D):
    dct1d = dct(np.eye(D), norm='ortho').T
    dct2d = np.empty((D,D,D,D))
    for i in range(D):
        for j in range(D):
            dct2d[i, j, :, :] = dct1d[i, :, np.newaxis] * dct1d[np.newaxis, j, :]

    return dct2d

In [ ]:
### 2D-DCTの基底を画像化する関数
#
def getDCTbasis2DImage(dct2d):

    gap = 2
    D = dct2d.shape[0]
    imgsize  = D * (D + gap) + gap
    img = np.zeros((imgsize, imgsize))

    for iy in range(D):
        lty = iy*(D + gap) + gap
        for ix in range(D):
            ltx = ix*(D + gap) + gap
            img[lty:lty+D, ltx:ltx+D] = dct2d[iy, ix, :, :]

    absmax = np.max(np.abs(img))
    img = img * 127.5 / absmax + 127.5

    return img

サンプル画像を入手します．

In [ ]:
# 画像を入手
!wget -nc https://www-tlab.math.ryukoku.ac.jp/~takataka/course/MVA/uni3.png

# 元はカラー画像なので，グレイスケールに変換
imgSrc = cv2.imread('uni3.png', cv2.IMREAD_GRAYSCALE)

# 表示
displayImage(imgSrc)
print(imgSrc.shape)

---
### 2次元DCT

2次元DCTの基底は，1次元のDCTの基底を組み合わせて作ることができます．ここでは式を表すのは省略して，縦横 $8\times 8$ の値に対応した2次元DCTの基底を画像として眺めてみることにします．

In [ ]:
# 8x8 の2次元DCTの基底を可視化
D = 8
dct2d = getDCTbasis2D(D)
print(f'dct2d.shape = {dct2d.shape}')
img = getDCTbasis2DImage(dct2d)

# そのままではサイズが小さいので，縦横に拡大して表示
img2 = cv2.resize(img, None, fx=6, fy=6, interpolation=cv2.INTER_NEAREST)
displayImage(img2)

上記のセルを実行すると，`dct2d` という4次元配列ができます．基底は，`dct2d[0, 0]` から `dct2d[D-1, D-1]` までの $D\times D$ 個（上の例では $64$ 個）から成り，そのそれぞれが $D\times D$ 個の値から成る2次元配列です．図の左上の灰色の正方形が `dct2d[0, 0]`，右上の縞模様が `dct2d[0, 7]`，右下の格子模様が `dct2d[7, 7]` に対応しています．それぞれの値は正の場合も負の場合もありますので，$0$がグレーに対応するようにして，正が白，負が黒になるように描いてあります．

2つの基底の値を表示させてみるとこんなん：

In [ ]:
print('##### dct2d[0, 0]')
print(dct2d[0, 0])
print()
print('##### dct2d[0, 1]')
print(dct2d[0, 1])

`dct2d[i, j]` の `i` が大きくなると縦方向の変化の周期が短くなり，`j` が大きくなると，横方向の変化の周期が短くなります．

---
### 画像に $8\times 8$ の2次元DCTを適用して展開係数を眺めてみる

画素数 $H \times W$ の画像を扱う場合，画像全体に対して $H \times W$ の2次元DCTを適用することもできますが，ここでは，画像を $8\times 8$ の小領域（ブロック）に分割して，それぞれに $8\times 8$ の2次元DCTを適用してみましょう．

In [ ]:
#@title `dct2d[iy, ix]` に対する展開係数の値を可視化してみる．以下の値をいろいろ変えてみよう．
iy =  0#@param {type:"integer"}
ix =  0#@param {type:"integer"}

D = 8
dct2d = getDCTbasis2D(D)

# (iy, ix) は以下を満たさないといけない
assert 0 <= ix and ix < D and 0 <= iy and iy < D

# 展開係数を計算（このやり方は計算効率が悪いので実用的ではない）
img = cv2.filter2D(imgSrc, cv2.CV_32F, dct2d[iy, ix])
img = cv2.resize(img, (imgSrc.shape[1]//D, imgSrc.shape[0]//D), interpolation = cv2.INTER_NEAREST)

# 展開係数の値を見やすくスケーリング
absmax = np.max(np.abs(img))
img = img / (2*absmax) + 0.5
img *= 255

# 縦横8倍に拡大して表示
displayImage(cv2.resize(img, None, fx=8, fy=8, interpolation=cv2.INTER_NEAREST))
print(f'dct2d[{iy},{ix}]に対する展開係数の値')

上記は，基底 `dct2d[iy, ix]` に対する展開係数の値をグレイスケール画像として可視化したものです．灰色が $0$ で，黒い画素は負の値，白い画素は正の値を表します．展開係数は元の画像の$8\times8$ の小領域毎に求まるので，そのまま可視化すれば縦横 $\frac{1}{8}$ の大きさになりますが，ここでは見やすくするために，元画像と同じ大きさに拡大して表示しています．


#### 問題1

次のことを考えて／調べてメモしておこう：

(1) `iy = 0` として，`ix` を徐々に大きくしていってみよう．このとき，展開係数の値の絶対値が大きいのは，元の画像のどんなところだろうか．元画像のその場所の画素値と基底の値にはどんな関係があるだろうか．

(2) `ix = 0` として，`iy` を徐々に大きくしていってみよう．このとき，（以下同文）．



### 2次元DCTを利用した画像圧縮

notebookAの「直交展開を利用したベクトルの近似」のセクションで，一部の展開係数のみを用いて元のベクトルを近似できること，それによってデータ圧縮が可能なことを説明しました．2次元DCTは，画像圧縮（画像のデータ圧縮）に利用することができます．実は，↑で出てきた $8\times 8$ の2次元DCTは，ファイル名の拡張子 `.jpg` や `.jpeg` でおなじみの，JPEG （注）と呼ばれる画像圧縮技術の中核として使用されています．実際の JPEG ではDCTの他にも様々な技術が組み合わされていますが，ここでは，画像圧縮の原理や考え方を理解することを目的として，2次元DCTのみを用いた「画像圧縮もどき」の実験をやってみましょう．

※ 注意: JPEG に関する Wikipedia の記事: https://ja.wikipedia.org/wiki/JPEG



グレイスケール画像に $8\times 8$ の2次元DCTを適用すると，$8\times 8$ 画素のブロックごとに $8\times 8 = 64$ 個の展開係数が得られます．これらの展開係数の値と `dct2d[0, 0]` から `dct2d[7, 7]` までの $64$ 個の基底をすべて用いれば元のブロックの画素値を再現できますが，すべては用いず，一部だけを用いると，元の画素値を近似することになります．
例えば，基底の図において左上にある `dct2d[0, 0]`, `dct2d[0, 1]`, `dct2d[1, 0]`, `dct2d[1, 1]` の4つに対応した展開係数のみで近似する場合，元の画素値では1ブロックを表現するのに $64$ 個の数値が必要だったところが，展開係数 $4$ 個で済むことになり，必要なデータ量を $1/16$ に削減できることになります（注）．

<span style="font-size: 75%">
※注: 画素値は $[0, 255]$ の整数値なので $8$ [bit] で表せますが，展開係数は実数なので，そのままでは，浮動小数点数として表すためにビット数がたくさん必要になってしまいます．実際のJPEGでは，展開係数の値を量子化して必要なビット数を削減します．さらに，一部の展開係数を捨てるのではなく，係数ごとに量子化ビット数を変えることで，圧縮効率と画質を両立させています．
</span>


というわけで，64個の展開係数のうち左上の $H\times H$ 個だけを使って画像を近似してみましょう．この場合，基底の図で右の方や下の方にあるような，縦横の画素値の変化が大きいような成分は捨てることになります．どれくらい元画像を近似できるでしょうか．

In [ ]:
### dct2d[0, 0] から dct2d[H-1, H-1] までの基底を使って画像を再構成する関数
#
def getReconstruction(imgSrc, dct2d, H):

    D = dct2d.shape[0]
    wSrc, hSrc = imgSrc.shape[1], imgSrc.shape[0]
    wDst, hDst = wSrc//D*D, hSrc//D*D
    imgDst = np.zeros((hDst, wDst))

    # このやり方は効率悪いので実用的ではない
    for iy in range(0, hSrc, D):
        for ix in range(0, wSrc, D):
            x = imgSrc[iy:iy+D, ix:ix+D].reshape(-1).astype(float)
            for jy in range(H):
                for jx in range(H):
                    coeff = x @ dct2d[jy, jx, :, :].reshape(-1)
                    imgDst[iy:iy+D, ix:ix+D] += coeff * dct2d[jy, jx, :, :]

    return imgDst

In [ ]:
#@title `dct2d[0, 0]` から `dct2d[H-1, H-1]` までを使って画像圧縮もどき
H = 1 #@param {type:"integer"}
D = 8

# H は以下を満たさないといけない
assert 1 <= H and H <= D

dct2d = getDCTbasis2D(D)
imgSrc = cv2.imread('uni3.png', cv2.IMREAD_GRAYSCALE)
imgDst = getReconstruction(imgSrc, dct2d, H)

# 得られた画像を表示
displayImage(imgDst)
print(f'H = {H}')
print()

# 一部拡大表示
img = cv2.resize(imgDst[0:100, 430:530], None, fx=4, fy=4, interpolation=cv2.INTER_NEAREST)
displayImage(img)

#### 問題2

`H` の値をいろいろ変えて結果を観察しよう．



ちなみに，$H=8$ とすると元画像が完全に再現されますが，それでもひげの周囲にもやもやとしたノイズが見えるでしょう．これは，最初から元画像に含まれているものです．実は元画像はディジタルカメラで撮影した画像をもとにしており，撮影して保存した時点で JPEG による圧縮がかかっています．元画像に含まれるこのようなノイズは，基底の図で右や下の方にある成分を粗く量子化したことによって発生したものです．蚊がたかっている様だということで「モスキートノイズ」と呼ばれたりします．

好きな画像で上記と同じ実験をやってみよう．

(1) 手元のPCに適当な画像を用意しましょう．ファイル名は空白や全角文字を含まないようにしましょう．

(2) 次のセル内をコメントにしたがって修正して実行し，適当な画像ファイルをアップロードしましょう．

In [ ]:
# ファイルをアップロード．このセルを実行して「ファイル選択」ボタンを押して...
from google.colab import files

if 1 == 0: # ← の 0 を 1 に修正
    uploaded = files.upload()

!ls -l

(3) `ls` コマンドの実行結果が表示されるので，アップロードした画像のファイル名を確認して，それを次のセルに書いて実行しましょう．

In [ ]:
fn = 'uni3.png' # ここにアップロードしたファイル名を書く

img = cv2.imread(fn, cv2.IMREAD_GRAYSCALE) # グレイスケール画像として読み込む
if img is None:
    print(f'{fn} を読み込めませんでした')

# 長い方の辺の長さが 640 になるようにリサイズ
h, w = img.shape
if w > h:
    w2 = 640
    h2 = int(640/w * h)
else:
    h2 = 640
    w2 = int(640/h * w)
img = cv2.resize(img, dsize=(w2, h2))

# 画像サイズが 8 の整数倍になるように一部切り捨て
img = img[:h2//8*8, :w2//8*8]

# 画像を表示
displayImage(img)

# リサイズした画像を hoge.png という名前で保存
cv2.imwrite('hoge.png', img)

! ls -l

(4) グレイスケールにしてリサイズした画像が `hoge.png` という名前で保存されたはずです．「画像圧縮もどき」のセルの中の次の行のファイル名を `hoge.png` に変更して実行してみましょう．

```
imgSrc = cv2.imread('uni3.png', cv2.IMREAD_GRAYSCALE)
```